# Wikidata: The state of the art (IMAGES)

This notebook consolidates all Wikidata processing for VKUS Viewer 

In [ ]:
!pip3 install SPARQLWrapper
!pip install pyvips pillow requests

In [ ]:
import os
os.environ["DYLD_LIBRARY_PATH"] = "/opt/homebrew/lib"

import json
import hashlib
import time
import requests
import re
import csv
import pyvips
import tempfile
import shutil
import urllib.parse
from datetime import datetime
from SPARQLWrapper import SPARQLWrapper, JSON

ENDPOINT_URL = "https://query.wikidata.org/sparql"
USER_AGENT = "Data on Paintings; Marian Dörk"
SAFE_COMMA = "，"  # fullwidth comma

CACHE_DIR = "wikidata_cache"
IMAGES_DIR = "vv/data/images"
SKIPPED_DIR = "vv/data/images/skipped"

os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(IMAGES_DIR, exist_ok=True)
os.makedirs(SKIPPED_DIR, exist_ok=True)

ENTITY_URL_TEMPLATE = "https://www.wikidata.org/wiki/Special:EntityData/{}.json"

# =============================================================================
# HELPER FUNCTIONS
# =============================================================================

def ts():
    return datetime.now().strftime("%H:%M:%S")

def query_hash(query_template: str) -> str:
    """Stable hash for a query string."""
    h = hashlib.sha1()
    h.update(query_template.encode("utf-8"))
    return h.hexdigest()[:12]

def get_sparql(query, max_retries=3):
    """Execute SPARQL query with caching and retry logic."""
    qhash = query_hash(query)
    cache_file = os.path.join(CACHE_DIR, f"{qhash}.json")
    
    if os.path.exists(cache_file):
        with open(cache_file, "r") as f:
            print(f"{ts()} | [cache] {qhash}")
            return json.load(f)
    
    sparql = SPARQLWrapper(ENDPOINT_URL, agent=USER_AGENT)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    for attempt in range(max_retries):
        try:
            print(f"{ts()} | [query] {qhash}...")
            
            # Get raw response and clean control characters before parsing
            response = sparql.query()
            raw_data = response.response.read().decode('utf-8')
            cleaned_data = re.sub(r'[\x00-\x1f\x7f-\x9f]', ' ', raw_data)
            result = json.loads(cleaned_data)
            
            with open(cache_file, "w") as f:
                json.dump(result, f)
            
            print(f"{ts()} | [saved] {qhash}")
            time.sleep(2)
            return result
        except Exception as e:
            if attempt < max_retries - 1:
                wait = (attempt + 1) * 5
                print(f"{ts()} | [retry] Attempt {attempt + 1} failed: {e}")
                print(f"{ts()} | [retry] Waiting {wait}s...")
                time.sleep(wait)
            else:
                raise e

def get_entity(qid, session=None):
    """Get entity data from cache or fetch from Wikidata."""
    cache_file = os.path.join(CACHE_DIR, f"{qid}.json")
    
    if os.path.exists(cache_file):
        with open(cache_file, "r") as f:
            return json.load(f)
    
    if session is None:
        session = requests.Session()
        session.headers.update({"User-Agent": USER_AGENT})
    
    url = ENTITY_URL_TEMPLATE.format(qid)
    resp = session.get(url, timeout=30)
    resp.raise_for_status()
    data = resp.json()
    entity = data.get("entities", {}).get(qid)
    
    if entity:
        with open(cache_file, "w") as f:
            json.dump(entity, f, ensure_ascii=False, indent=2)
    
    return entity

def get_label(qid, lang="en", session=None):
    """Get label for a Q-ID from cached entity data."""
    entity = get_entity(qid, session)
    if entity is None:
        return qid
    
    labels = entity.get("labels", {})
    if lang in labels:
        return labels[lang]["value"]
    elif "en" in labels:
        return labels["en"]["value"]
    elif labels:
        return list(labels.values())[0]["value"]
    return qid

def get_claim_values(entity, property_id):
    """Extract Q-IDs from a property's claims."""
    if entity is None:
        return []
    
    claims = entity.get("claims", {}).get(property_id, [])
    values = []
    for claim in claims:
        mainsnak = claim.get("mainsnak", {})
        datavalue = mainsnak.get("datavalue", {})
        if datavalue.get("type") == "wikibase-entityid":
            values.append(datavalue["value"]["id"])
    return values

def format_time_remaining(seconds):
    """Format seconds into human-readable string."""
    if seconds < 60:
        return f"{seconds:.0f}s"
    elif seconds < 3600:
        return f"{seconds / 60:.0f}m"
    else:
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        return f"{hours}h {minutes}m"


In [ ]:
# =============================================================================
# 1. INITIAL WIKIDATA QUERY FOR IDS
# =============================================================================

painting_qids_query = """
SELECT DISTINCT ?painting WHERE {
  ?painting wdt:P31/wdt:P279* wd:Q3305213;
            wdt:P18 ?image;
            wdt:P571 ?inception.
  ?painting wikibase:sitelinks ?sitelinks.
  FILTER(?sitelinks >= 3)
}
"""

# painting_qids_query = """
# SELECT DISTINCT ?painting WHERE {
#   ?painting wdt:P31/wdt:P279* wd:Q3305213 ;
#             wdt:P18 ?image ;
#             wdt:P571 ?inception .
#   ?painting wikibase:sitelinks ?paintingSitelinks .
#   OPTIONAL {
#     ?painting wdt:P170 ?creator .
#     ?creator wikibase:sitelinks ?creatorSitelinks .
#   }
#   FILTER(?paintingSitelinks >= 10 || ?creatorSitelinks >= 100)
# }
# """


print(f"{ts()} | Querying Wikidata for paintings...")

result = get_sparql(painting_qids_query)
painting_rows = result["results"]["bindings"]
painting_qids = [row["painting"]["value"].split("/")[-1] for row in painting_rows]

print(f"{ts()} | Retrieved {len(painting_qids):,} unique paintings")
print(f"{ts()} | Criteria: has image, has inception date, 3+ sitelinks")

In [ ]:
# =============================================================================
# 2. FETCH WIKIDATA ENTITY JSONs per Q-ID
# =============================================================================

print(f"\nFetching entity data for {len(painting_qids):,} paintings...")

session = requests.Session()
session.headers.update({"User-Agent": USER_AGENT})

# Adaptive retry timing
retry_sleep = 10.0
min_retry_sleep = 10.0
max_retry_sleep = 3600.0
consecutive_successes = 0

# Progress tracking
total = len(painting_qids)
fetched = 0
start_time = time.time()

for i, qid in enumerate(painting_qids):
    cache_file = os.path.join(CACHE_DIR, f"{qid}.json")
    
    # Already cached
    if os.path.exists(cache_file):
        consecutive_successes += 1
        if consecutive_successes >= 2:
            retry_sleep = max(retry_sleep / 2, min_retry_sleep)
            consecutive_successes = 0
        continue
    
    fetched += 1
    
    # Progress with time estimate
    if fetched % 50 == 0 or fetched == 1:
        elapsed = time.time() - start_time
        rate = fetched / elapsed if elapsed > 0 else 0
        
        remaining_to_check = total - (i + 1)
        checked_so_far = i + 1
        cache_hit_rate = 1 - (fetched / checked_so_far) if checked_so_far > 0 else 0
        estimated_remaining_fetches = remaining_to_check * (1 - cache_hit_rate)
        estimated_seconds = estimated_remaining_fetches / rate if rate > 0 else 0
        
        print(f"{ts()} | [progress] {fetched:,} fetched @ {rate:.1f}/s | {i + 1:,}/{total:,} checked | ~{format_time_remaining(estimated_seconds)} remaining")
    
    # Fetch with retry
    url = ENTITY_URL_TEMPLATE.format(qid)
    
    while True:
        try:
            resp = session.get(url, timeout=30)
            resp.raise_for_status()
            data = resp.json()
            entity = data.get("entities", {}).get(qid)
            
            if not entity:
                print(f"{ts()} | {qid} | empty entity")
                break
            
            with open(cache_file, "w") as f:
                json.dump(entity, f, ensure_ascii=False, indent=2)
            
            consecutive_successes += 1
            if consecutive_successes >= 2:
                retry_sleep = max(retry_sleep / 2, min_retry_sleep)
                consecutive_successes = 0
            break
            
        except Exception as e:
            consecutive_successes = 0
            retry_sleep = min(retry_sleep * 2, max_retry_sleep)
            msg = str(e).split("\n")[0][:80]
            print(f"{ts()} | {qid} | retry in {retry_sleep:.1f}s ({msg})")
            time.sleep(retry_sleep)

elapsed_total = time.time() - start_time
print(f"\n{ts()} | [done] Fetched {fetched:,} new entities in {format_time_remaining(elapsed_total)}")


In [ ]:
# =============================================================================
# 3. DOWNLOAD IMAGES
# =============================================================================

MAX_DIM = 5000
os.makedirs(IMAGES_DIR, exist_ok=True)

session = requests.Session()
session.headers.update({"User-Agent": USER_AGENT})

print(f"{ts()} | Checking which images need to be downloaded...")

# ---------------------------------------------------------------------------
# Pre-check: identify paintings that need downloading
# ---------------------------------------------------------------------------

to_download = []

for qid in painting_qids:
    out_path = os.path.join(IMAGES_DIR, f"{qid}.jpg")
    skipped_path = os.path.join(SKIPPED_DIR, f"{qid}.jpg")
    
    # Already exists in images/
    if os.path.exists(out_path) and os.path.getsize(out_path) > 0:
        continue
    
    # Exists in skipped/ - will be restored
    if os.path.exists(skipped_path) and os.path.getsize(skipped_path) > 0:
        shutil.move(skipped_path, out_path)
        continue
    
    to_download.append(qid)

print(f"{ts()} | Total paintings: {len(painting_qids):,}")
print(f"{ts()} | Already downloaded: {len(painting_qids) - len(to_download):,}")
print(f"{ts()} | Remaining to download: {len(to_download):,}")

if not to_download:
    print(f"{ts()} | [done] All images already downloaded")
else:
    # ---------------------------------------------------------------------------
    # Adaptive rate limiting
    # ---------------------------------------------------------------------------
    SLEEP_STEPS = [1, 5, 15, 30, 60, 120, 300, 600]
    current_step = 0
    success_streak = 0
    STREAK_THRESHOLD = 100

    def get_current_sleep():
        return SLEEP_STEPS[current_step]

    def wait_with_backoff():
        """Sleep for current delay, used between requests."""
        sleep_time = get_current_sleep()
        if current_step > 0:
            print(f"{ts()} | ... sleeping {sleep_time}s (rate limited, step {current_step}/{len(SLEEP_STEPS)-1})")
        time.sleep(sleep_time)

    def get_image_filename(entity):
        """Extract image filename from P18 claim."""
        for c in entity.get("claims", {}).get("P18", []):
            dv = c.get("mainsnak", {}).get("datavalue")
            if dv and dv.get("type") == "string":
                filename = dv.get("value", "")
                if filename:
                    return filename.replace(" ", "_")
        return None

    def refetch_entity(qid):
        """Force re-fetch entity from Wikidata (bypass cache)."""
        cache_file = os.path.join(CACHE_DIR, f"{qid}.json")
        
        # Remove cached version
        if os.path.exists(cache_file):
            os.remove(cache_file)
        
        # Fetch fresh
        url = ENTITY_URL_TEMPLATE.format(qid)
        try:
            resp = session.get(url, timeout=30)
            resp.raise_for_status()
            data = resp.json()
            entity = data.get("entities", {}).get(qid)
            
            if entity:
                with open(cache_file, "w") as f:
                    json.dump(entity, f, ensure_ascii=False)
            
            return entity
        except Exception as e:
            print(f"{ts()} | {qid} | refetch ERROR ({str(e)[:80]})")
            return None

    # ---------------------------------------------------------------------------
    # Download remaining images
    # ---------------------------------------------------------------------------
    total = len(to_download)
    downloaded = 0
    errors = 0
    no_image = 0
    start_time = time.time()

    print(f"\n{ts()} | Starting download of {total:,} images...")

    for i, qid in enumerate(to_download):
        out_path = os.path.join(IMAGES_DIR, f"{qid}.jpg")

        # Load entity from cache
        entity = get_entity(qid, session)
        
        if entity is None:
            errors += 1
            continue

        # Extract image filename (P18)
        filename = get_image_filename(entity)
        
        # If no filename, try re-fetching entity
        if not filename:
            print(f"{ts()} | {qid} | No image filename, re-fetching entity...")
            entity = refetch_entity(qid)
            if entity:
                filename = get_image_filename(entity)
        
        if not filename:
            print(f"{ts()} | {qid} | SKIP (no valid image filename)")
            no_image += 1
            continue

        image_url = "https://commons.wikimedia.org/wiki/Special:FilePath/" + urllib.parse.quote(filename)

        # Download → resize → JPEG (with retry on 429)
        max_retries = 10
        retry_count = 0
        
        while retry_count < max_retries:
            try:
                wait_with_backoff()
                
                with session.get(image_url, stream=True, timeout=30) as resp:
                    resp.raise_for_status()

                    with tempfile.NamedTemporaryFile(delete=False) as tmp:
                        for chunk in resp.iter_content(8192):
                            if chunk:
                                tmp.write(chunk)
                        tmp_path = tmp.name

                img = pyvips.Image.new_from_file(tmp_path, access="sequential")

                # Downscale if image area exceeds MAX_DIM * MAX_DIM
                max_area = MAX_DIM * MAX_DIM
                area = img.width * img.height
                scale = min(1.0, (max_area / area) ** 0.5)
                if scale < 1.0:
                    img = img.resize(scale)

                img.write_to_file(out_path, Q=90, strip=True)
                os.remove(tmp_path)
                
                downloaded += 1
                
                # Progress with time estimate
                if downloaded % 25 == 0 or downloaded == 1:
                    elapsed = time.time() - start_time
                    rate = downloaded / elapsed if elapsed > 0 else 0
                    remaining = total - (i + 1)
                    estimated_seconds = remaining / rate if rate > 0 else 0
                    
                    print(f"{ts()} | [progress] {downloaded:,}/{total:,} downloaded @ {rate:.2f}/s | ~{format_time_remaining(estimated_seconds)} remaining")
                
                # Success: track streak and step down cautiously
                if current_step > 0:
                    success_streak += 1
                    if success_streak >= STREAK_THRESHOLD:
                        current_step -= 1
                        success_streak = 0
                        print(f"{ts()} | ↓ stepping down to {get_current_sleep()}s delay")
                
                break  # exit retry loop on success

            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 429:
                    retry_count += 1
                    success_streak = 0
                    if current_step < len(SLEEP_STEPS) - 1:
                        current_step += 1
                    print(f"{ts()} | {qid} | 429 rate limited, retry {retry_count}/{max_retries}, step up → {get_current_sleep()}s")
                    if retry_count >= max_retries:
                        print(f"{ts()} | {qid} | ERROR (max retries exceeded)")
                        errors += 1
                elif e.response.status_code == 404:
                    print(f"{ts()} | {qid} | SKIP (image not found: {filename[:50]}...)")
                    no_image += 1
                    break
                else:
                    print(f"{ts()} | {qid} | ERROR ({str(e)[:80]})")
                    errors += 1
                    break
                    
            except Exception as e:
                print(f"{ts()} | {qid} | ERROR ({str(e)[:80]})")
                if os.path.exists(out_path):
                    os.remove(out_path)
                errors += 1
                break

    elapsed_total = time.time() - start_time
    print(f"\n{ts()} | [done] Downloaded {downloaded:,}, no image {no_image:,}, errors {errors:,} in {format_time_remaining(elapsed_total)}")

In [ ]:
# =============================================================================
# 4. BUILD RECORDS FROM PAINTING ENTITIES
# =============================================================================

# ---------------------------------------------------------------------------
# Helper functions
# ---------------------------------------------------------------------------

def normalize_value(value):
    if not value:
        return ""
    return value.replace(",", SAFE_COMMA).strip()

def add_keyword(rec, prefix, qid):
    """Add a keyword in format 'prefix:Q12345' to the record's keywords set."""
    if not prefix or not qid:
        return
    if not qid.startswith("Q"):
        return
    rec["keywords"].add(f"{prefix}:{qid}")

def add_column_value(rec, field, qids):
    if not qids:
        return
    existing = [v.strip() for v in rec[field].split(",") if v.strip()]
    for q in qids:
        if q not in existing:
            existing.append(q)
    rec[field] = ", ".join(existing)

def parse_wikidata_time(timeval):
    """Convert Wikidata time string like '+1505-01-01T00:00:00Z' to int year."""
    if not timeval:
        return None
    m = re.match(r'^\+?(\d+)', timeval)
    if m:
        return int(m.group(1))
    return None

def resolve_claim_qids(entity, prop):
    """Return list of QIDs for a given property (Pxxx) from claims."""
    out = []
    for c in entity.get("claims", {}).get(prop, []):
        dv = c.get("mainsnak", {}).get("datavalue")
        if dv and dv.get("type") == "wikibase-entityid":
            out.append(f"Q{dv['value']['numeric-id']}")
    return out

def resolve_claim_values(entity, prop):
    """Return list of string values (for P18, etc.) from claims."""
    out = []
    for c in entity.get("claims", {}).get(prop, []):
        dv = c.get("mainsnak", {}).get("datavalue")
        if not dv:
            continue
        if dv["type"] == "string":
            out.append(dv["value"])
        elif dv["type"] == "time":
            out.append(dv["value"]["time"])
    return out

def clean_movement_label(label):
    """Clean up movement labels by removing ' painting' suffix."""
    if not label:
        return label
    if label.lower().endswith(" painting"):
        label = label[:-9]
    return label.title()

# ---------------------------------------------------------------------------
# Pre-scan: Build movement QID harmonization map
# ---------------------------------------------------------------------------

print(f"{ts()} | Pre-scanning movements for harmonization...")

# First, collect all unique movement QIDs from paintings and artists
all_movement_qids = set()

for i, qid in enumerate(painting_qids):
    if (i + 1) % 100000 == 0:
        print(f"{ts()} | [scanning paintings] {i + 1:,}/{len(painting_qids):,}")
    
    entity = get_entity(qid)
    if entity is None:
        continue
    
    all_movement_qids.update(resolve_claim_qids(entity, "P135"))

print(f"{ts()} | Unique movement QIDs from paintings: {len(all_movement_qids):,}")

# We'll also need to scan artist movements later, but first let's fetch labels
# for what we have so far

# Fetch movement entities if not cached
movements_to_fetch = [qid for qid in all_movement_qids if not os.path.exists(os.path.join(CACHE_DIR, f"{qid}.json"))]

if movements_to_fetch:
    print(f"{ts()} | Fetching {len(movements_to_fetch):,} movement entities...")
    session = requests.Session()
    session.headers.update({"User-Agent": USER_AGENT})
    
    for i, qid in enumerate(movements_to_fetch):
        try:
            get_entity(qid, session)
        except Exception as e:
            print(f"{ts()} | {qid} | ERROR ({str(e)[:80]})")

# Build harmonization map: movement QID -> canonical QID
# Multiple QIDs with the same cleaned label map to one canonical QID
movement_label_to_canonical = {}  # cleaned label (lowercase) -> canonical QID
movement_qid_to_canonical = {}    # any movement QID -> canonical QID

for qid in all_movement_qids:
    entity = get_entity(qid)
    if not entity:
        continue
    
    # Get label
    labels = entity.get("labels", {})
    raw_label = ""
    if "en" in labels:
        raw_label = labels["en"].get("value", "")
    elif labels:
        raw_label = list(labels.values())[0].get("value", "")
    
    if not raw_label:
        movement_qid_to_canonical[qid] = qid  # keep as-is
        continue
    
    cleaned = clean_movement_label(raw_label)
    cleaned_lower = cleaned.lower()
    
    if cleaned_lower in movement_label_to_canonical:
        # Map to existing canonical QID
        canonical_qid = movement_label_to_canonical[cleaned_lower]
        movement_qid_to_canonical[qid] = canonical_qid
        print(f"{ts()} | Harmonizing: {qid} ({raw_label}) -> {canonical_qid} ({cleaned})")
    else:
        # This becomes the canonical QID for this label
        movement_label_to_canonical[cleaned_lower] = qid
        movement_qid_to_canonical[qid] = qid

print(f"{ts()} | Movement QIDs: {len(all_movement_qids):,}")
print(f"{ts()} | Unique labels after harmonization: {len(movement_label_to_canonical):,}")

def harmonize_movement_qids(qids):
    """Map a list of movement QIDs to their canonical forms, removing duplicates."""
    canonical = set()
    for qid in qids:
        canonical.add(movement_qid_to_canonical.get(qid, qid))
    return list(canonical)

# ---------------------------------------------------------------------------
# Build records from painting_qids (on demand loading)
# ---------------------------------------------------------------------------

print(f"\n{ts()} | Building records from {len(painting_qids):,} paintings...")

# Check for duplicates in painting_qids
unique_qids = set(painting_qids)
if len(unique_qids) != len(painting_qids):
    print(f"{ts()} | WARNING: {len(painting_qids) - len(unique_qids):,} duplicate Q-IDs in painting_qids")

records = {}
linked_qids = set()
artist_qids = set()
start_time = time.time()
total = len(painting_qids)

# Counters for transparency
skipped_no_entity = 0
skipped_no_id = 0
skipped_duplicate_uri = 0
records_no_year = 0
records_no_keywords = 0
processed = 0

for i, qid in enumerate(painting_qids):
    if (i + 1) % 2000 == 0:
        elapsed = time.time() - start_time
        rate = (i + 1) / elapsed if elapsed > 0 else 0
        remaining = (total - (i + 1)) / rate if rate > 0 else 0
        print(f"{ts()} | [building] {i + 1:,}/{total:,} @ {rate:.0f}/s | records: {len(records):,} | ~{format_time_remaining(remaining)} remaining")
    
    entity = get_entity(qid)
    
    if entity is None:
        skipped_no_entity += 1
        continue
    
    if "id" not in entity:
        skipped_no_id += 1
        continue
    
    entity_id = entity["id"]
    if entity_id != qid:
        print(f"{ts()} | WARNING: Requested {qid} but got entity {entity_id}")
    
    uri = f"https://www.wikidata.org/wiki/{qid}"
    
    if uri in records:
        skipped_duplicate_uri += 1
        continue

    # Get title with fallback
    title = ""
    labels = entity.get("labels", {})
    if "en" in labels:
        title = labels["en"].get("value", "")
    elif labels:
        title = list(labels.values())[0].get("value", "")

    rec = {
        "id": qid,
        "keywords": set(),
        "year": "",
        "_title": title,
        "_creation": "",
        "_artist": "",
        "_country": "",
        "_depicts": "",
        "_genre": "",
        "_material": "",
        "_movement": "",
        "_museum": ""
    }

    # Populate claims and collect linked Q-IDs
    for prop, field, kw_prefix in [
        ("P170", "_artist", "artist"),
        ("P195", "_museum", "museum"),
        ("P180", "_depicts", "depicts"),
        ("P186", "_material", "material"),
        ("P136", "_genre", "genre"),
        ("P135", "_movement", "movement"),
    ]:
        qids_list = resolve_claim_qids(entity, prop)
        
        # Harmonize movement QIDs
        if prop == "P135":
            qids_list = harmonize_movement_qids(qids_list)
        
        linked_qids.update(qids_list)
        
        if prop == "P170":
            artist_qids.update(qids_list)
        
        if field in ["_depicts", "_material", "_genre", "_movement"]:
            add_column_value(rec, field, qids_list)
        else:
            rec[field] = ", ".join(qids_list)
        
        for q in qids_list:
            add_keyword(rec, kw_prefix, q)

    # Creation dates
    years = [parse_wikidata_time(t) for t in resolve_claim_values(entity, "P571") if parse_wikidata_time(t)]
    if years:
        years = sorted(set(years))
        if len(years) > 1:
            rec["_creation"] = f"{years[0]}-{years[-1]}"
            decade_base = years[0]
        else:
            rec["_creation"] = str(years[0])
            decade_base = years[0]
        rec["year"] = str((decade_base // 10) * 10)
    else:
        records_no_year += 1
    
    if not rec["keywords"]:
        records_no_keywords += 1

    records[uri] = rec
    processed += 1

elapsed_total = time.time() - start_time

print(f"\n{ts()} | [done] Building complete in {format_time_remaining(elapsed_total)}")
print(f"{ts()} | ")
print(f"{ts()} | Input:     {len(painting_qids):,} painting Q-IDs")
print(f"{ts()} | Unique:    {len(unique_qids):,} unique Q-IDs")
print(f"{ts()} | Processed: {processed:,} entities")
print(f"{ts()} | Output:    {len(records):,} records built")
print(f"{ts()} | ")
print(f"{ts()} | Skipped:")
print(f"{ts()} |   - No entity in cache:  {skipped_no_entity:,}")
print(f"{ts()} |   - No 'id' in entity:   {skipped_no_id:,}")
print(f"{ts()} |   - Duplicate URI:       {skipped_duplicate_uri:,}")
print(f"{ts()} | ")
print(f"{ts()} | Unaccounted: {len(painting_qids) - skipped_no_entity - skipped_no_id - skipped_duplicate_uri - len(records):,}")
print(f"{ts()} | ")
print(f"{ts()} | Records with issues:")
print(f"{ts()} |   - No year:     {records_no_year:,}")
print(f"{ts()} |   - No keywords: {records_no_keywords:,}")
print(f"{ts()} | ")
print(f"{ts()} | Linked entities found: {len(linked_qids):,}")
print(f"{ts()} | Artists found: {len(artist_qids):,}")

# ---------------------------------------------------------------------------
# Pre-fetch linked entities (for label resolution and artist data)
# ---------------------------------------------------------------------------

print(f"\n{ts()} | Checking linked entities...")

to_fetch = [qid for qid in linked_qids if not os.path.exists(os.path.join(CACHE_DIR, f"{qid}.json"))]

print(f"{ts()} | Linked entities: {len(linked_qids):,}")
print(f"{ts()} | Already cached:  {len(linked_qids) - len(to_fetch):,}")
print(f"{ts()} | To fetch:        {len(to_fetch):,}")

if to_fetch:
    session = requests.Session()
    session.headers.update({"User-Agent": USER_AGENT})
    
    start_time = time.time()
    fetch_errors = 0
    
    for i, qid in enumerate(to_fetch):
        if (i + 1) % 50 == 0 or i == 0:
            elapsed = time.time() - start_time
            rate = (i + 1) / elapsed if elapsed > 0 else 0
            remaining = (len(to_fetch) - (i + 1)) / rate if rate > 0 else 0
            print(f"{ts()} | [fetch] {i + 1:,}/{len(to_fetch):,} @ {rate:.1f}/s | ~{format_time_remaining(remaining)} remaining")
        
        try:
            get_entity(qid, session)
        except Exception as e:
            fetch_errors += 1
            print(f"{ts()} | {qid} | ERROR ({str(e)[:80]})")
    
    elapsed_total = time.time() - start_time
    print(f"\n{ts()} | [done] Fetched {len(to_fetch):,} linked entities in {format_time_remaining(elapsed_total)}")
    if fetch_errors:
        print(f"{ts()} | Fetch errors: {fetch_errors:,}")
else:
    print(f"{ts()} | [done] All linked entities already cached")

# ---------------------------------------------------------------------------
# Build artist data lookup (movements + birthplace → country)
# ---------------------------------------------------------------------------

print(f"\n{ts()} | Building artist data (movements + countries)...")

artist_movements = {}
artist_countries = {}
birthplace_qids = set()

# Also scan artist movements and add to harmonization map
artist_movement_qids = set()

for artist_qid in artist_qids:
    artist_entity = get_entity(artist_qid)
    if not artist_entity:
        continue
    
    # Get movements (P135)
    movements = resolve_claim_qids(artist_entity, "P135")
    if movements:
        artist_movement_qids.update(movements)
    
    # Get birthplace (P19)
    birthplaces = resolve_claim_qids(artist_entity, "P19")
    if birthplaces:
        birthplace_qids.update(birthplaces)

# Fetch any new movement entities from artists
new_movement_qids = artist_movement_qids - all_movement_qids
if new_movement_qids:
    print(f"{ts()} | Found {len(new_movement_qids):,} new movement QIDs from artists")
    
    movements_to_fetch = [qid for qid in new_movement_qids if not os.path.exists(os.path.join(CACHE_DIR, f"{qid}.json"))]
    
    if movements_to_fetch:
        session = requests.Session()
        session.headers.update({"User-Agent": USER_AGENT})
        
        for qid in movements_to_fetch:
            try:
                get_entity(qid, session)
            except Exception as e:
                print(f"{ts()} | {qid} | ERROR ({str(e)[:80]})")
    
    # Add to harmonization map
    for qid in new_movement_qids:
        if qid in movement_qid_to_canonical:
            continue
        
        entity = get_entity(qid)
        if not entity:
            movement_qid_to_canonical[qid] = qid
            continue
        
        labels = entity.get("labels", {})
        raw_label = ""
        if "en" in labels:
            raw_label = labels["en"].get("value", "")
        elif labels:
            raw_label = list(labels.values())[0].get("value", "")
        
        if not raw_label:
            movement_qid_to_canonical[qid] = qid
            continue
        
        cleaned = clean_movement_label(raw_label)
        cleaned_lower = cleaned.lower()
        
        if cleaned_lower in movement_label_to_canonical:
            canonical_qid = movement_label_to_canonical[cleaned_lower]
            movement_qid_to_canonical[qid] = canonical_qid
            print(f"{ts()} | Harmonizing (artist): {qid} ({raw_label}) -> {canonical_qid} ({cleaned})")
        else:
            movement_label_to_canonical[cleaned_lower] = qid
            movement_qid_to_canonical[qid] = qid

# Now build artist_movements with harmonized QIDs
for artist_qid in artist_qids:
    artist_entity = get_entity(artist_qid)
    if not artist_entity:
        continue
    
    movements = resolve_claim_qids(artist_entity, "P135")
    if movements:
        harmonized = harmonize_movement_qids(movements)
        artist_movements[artist_qid] = harmonized
        linked_qids.update(harmonized)

print(f"{ts()} | Artists with movements: {len(artist_movements):,}")
print(f"{ts()} | Birthplaces found: {len(birthplace_qids):,}")

# ---------------------------------------------------------------------------
# Fetch birthplace entities to get their countries
# ---------------------------------------------------------------------------

print(f"\n{ts()} | Fetching birthplace entities...")

to_fetch_birthplaces = [qid for qid in birthplace_qids if not os.path.exists(os.path.join(CACHE_DIR, f"{qid}.json"))]

if to_fetch_birthplaces:
    session = requests.Session()
    session.headers.update({"User-Agent": USER_AGENT})
    
    start_time = time.time()
    
    for i, qid in enumerate(to_fetch_birthplaces):
        if (i + 1) % 50 == 0 or i == 0:
            elapsed = time.time() - start_time
            rate = (i + 1) / elapsed if elapsed > 0 else 0
            remaining = (len(to_fetch_birthplaces) - (i + 1)) / rate if rate > 0 else 0
            print(f"{ts()} | [fetch] {i + 1:,}/{len(to_fetch_birthplaces):,} @ {rate:.1f}/s | ~{format_time_remaining(remaining)} remaining")
        
        try:
            get_entity(qid, session)
        except Exception as e:
            print(f"{ts()} | {qid} | ERROR ({str(e)[:80]})")
    
    print(f"{ts()} | [done] Fetched {len(to_fetch_birthplaces):,} birthplace entities")
else:
    print(f"{ts()} | [done] All birthplace entities already cached")

# ---------------------------------------------------------------------------
# Build birthplace → country mapping (using SPARQL for current country)
# ---------------------------------------------------------------------------

print(f"\n{ts()} | Resolving birthplace → current country...")

def get_current_country_for_place(place_qid):
    """
    Get the current country for a place using SPARQL.
    Looks for country (P17) without an end time (P582).
    """
    query = f"""
    SELECT ?country WHERE {{
      wd:{place_qid} p:P17 ?stmt .
      ?stmt ps:P17 ?country .
      FILTER NOT EXISTS {{ ?stmt pq:P582 ?endTime }}
    }}
    LIMIT 1
    """
    
    try:
        result = get_sparql(query)
        bindings = result.get("results", {}).get("bindings", [])
        if bindings:
            country_uri = bindings[0].get("country", {}).get("value", "")
            if country_uri:
                return country_uri.split("/")[-1]
    except Exception as e:
        # Fallback: just use first country from entity
        pass
    
    return None

# First try to get current country via SPARQL for each unique birthplace
# To avoid too many queries, batch them

birthplace_to_country = {}
country_qids = set()

# Build a single SPARQL query for all birthplaces
print(f"{ts()} | Querying current countries for {len(birthplace_qids):,} birthplaces...")

# Split into batches to avoid query timeout
BATCH_SIZE = 100
birthplace_list = list(birthplace_qids)

for batch_start in range(0, len(birthplace_list), BATCH_SIZE):
    batch = birthplace_list[batch_start:batch_start + BATCH_SIZE]
    
    if (batch_start // BATCH_SIZE) % 10 == 0:
        print(f"{ts()} | [batch] {batch_start:,}/{len(birthplace_list):,}")
    
    # Build VALUES clause for batch query
    values_clause = " ".join([f"wd:{qid}" for qid in batch])
    
    query = f"""
    SELECT ?place ?country WHERE {{
      VALUES ?place {{ {values_clause} }}
      ?place p:P17 ?stmt .
      ?stmt ps:P17 ?country .
      FILTER NOT EXISTS {{ ?stmt pq:P582 ?endTime }}
    }}
    """
    
    try:
        result = get_sparql(query)
        bindings = result.get("results", {}).get("bindings", [])
        
        for b in bindings:
            place_uri = b.get("place", {}).get("value", "")
            country_uri = b.get("country", {}).get("value", "")
            
            if place_uri and country_uri:
                place_qid = place_uri.split("/")[-1]
                country_qid = country_uri.split("/")[-1]
                
                # Only take first result per place
                if place_qid not in birthplace_to_country:
                    birthplace_to_country[place_qid] = country_qid
                    country_qids.add(country_qid)
    
    except Exception as e:
        print(f"{ts()} | Batch query error: {str(e)[:80]}")

# For any birthplaces not resolved, fall back to cached entity data
fallback_count = 0
for bp_qid in birthplace_qids:
    if bp_qid in birthplace_to_country:
        continue
    
    bp_entity = get_entity(bp_qid)
    if not bp_entity:
        continue
    
    countries = resolve_claim_qids(bp_entity, "P17")
    if countries:
        birthplace_to_country[bp_qid] = countries[0]
        country_qids.add(countries[0])
        fallback_count += 1

print(f"{ts()} | Birthplaces with current country: {len(birthplace_to_country):,}")
print(f"{ts()} | Resolved via SPARQL: {len(birthplace_to_country) - fallback_count:,}")
print(f"{ts()} | Fallback to first country: {fallback_count:,}")
print(f"{ts()} | Unique countries: {len(country_qids):,}")

# Add countries to linked_qids for label resolution
linked_qids.update(country_qids)

# ---------------------------------------------------------------------------
# Build artist → country mapping
# ---------------------------------------------------------------------------

for artist_qid in artist_qids:
    artist_entity = get_entity(artist_qid)
    if not artist_entity:
        continue
    
    birthplaces = resolve_claim_qids(artist_entity, "P19")
    if birthplaces and birthplaces[0] in birthplace_to_country:
        artist_countries[artist_qid] = birthplace_to_country[birthplaces[0]]

print(f"{ts()} | Artists with country: {len(artist_countries):,}")

# ---------------------------------------------------------------------------
# Enrich paintings with artist movements and countries
# ---------------------------------------------------------------------------

print(f"\n{ts()} | Enriching paintings with artist data...")

enriched_movements = 0
enriched_countries = 0

for uri, rec in records.items():
    # Get artist QIDs from the record
    artist_str = rec.get("_artist", "")
    if not artist_str:
        continue
    
    painting_artist_qids = [q.strip() for q in artist_str.split(",") if q.strip().startswith("Q")]
    
    # Enrich movements (only if painting has none)
    if not rec["_movement"]:
        inherited_movements = set()
        for artist_qid in painting_artist_qids:
            if artist_qid in artist_movements:
                inherited_movements.update(artist_movements[artist_qid])
        
        if inherited_movements:
            add_column_value(rec, "_movement", list(inherited_movements))
            for m in inherited_movements:
                add_keyword(rec, "movement", m)
            enriched_movements += 1
    
    # Enrich countries (always, from first artist with country)
    if not rec["_country"]:
        for artist_qid in painting_artist_qids:
            if artist_qid in artist_countries:
                country_qid = artist_countries[artist_qid]
                rec["_country"] = country_qid
                add_keyword(rec, "country", country_qid)
                enriched_countries += 1
                break  # take first artist's country

print(f"{ts()} | Enriched {enriched_movements:,} paintings with artist movements")
print(f"{ts()} | Enriched {enriched_countries:,} paintings with artist countries")

# ---------------------------------------------------------------------------
# Fetch any new linked entities (movements + countries)
# ---------------------------------------------------------------------------

new_to_fetch = [qid for qid in linked_qids if not os.path.exists(os.path.join(CACHE_DIR, f"{qid}.json"))]

if new_to_fetch:
    print(f"\n{ts()} | Fetching {len(new_to_fetch):,} new entities (movements/countries)...")
    
    session = requests.Session()
    session.headers.update({"User-Agent": USER_AGENT})
    
    for i, qid in enumerate(new_to_fetch):
        try:
            get_entity(qid, session)
        except Exception as e:
            print(f"{ts()} | {qid} | ERROR ({str(e)[:80]})")
    
    print(f"{ts()} | [done]")

# ---------------------------------------------------------------------------
# Final stats
# ---------------------------------------------------------------------------

paintings_with_movement = sum(1 for rec in records.values() if rec["_movement"])
paintings_with_country = sum(1 for rec in records.values() if rec["_country"])

print(f"\n{ts()} | Final stats:")
print(f"{ts()} |   - Paintings with movements: {paintings_with_movement:,} / {len(records):,} ({100*paintings_with_movement/len(records):.1f}%)")
print(f"{ts()} |   - Paintings with country:   {paintings_with_country:,} / {len(records):,} ({100*paintings_with_country/len(records):.1f}%)")

In [ ]:
# =============================================================================
# 5. WRITE CSV (with claim labels & logging)
# =============================================================================

os.makedirs("vv/data", exist_ok=True)
csv_path = os.path.join("vv/data", "data.csv")

# Remove existing CSV if present
if os.path.exists(csv_path):
    os.remove(csv_path)
    print(f"{ts()} | Removed existing {csv_path}")

FIELDNAMES = [
    "id",
    "keywords",
    "year",
    "_title",
    "_creation",
    "_artist",
    "_country",
    "_depicts",
    "_genre",
    "_material",
    "_movement",
    "_museum",
]

# ---------------------------------------------------------------------------
# Helper function for cleaning movement labels (if not already defined)
# ---------------------------------------------------------------------------

def clean_movement_label(label):
    """Clean up movement labels by removing ' painting' suffix."""
    if not label:
        return label
    if label.lower().endswith(" painting"):
        label = label[:-9]
    return label.title()

# ---------------------------------------------------------------------------
# Pre-load all linked entity labels into memory
# ---------------------------------------------------------------------------

print(f"{ts()} | Pre-loading labels for {len(linked_qids):,} linked entities...")

labels_cache = {}
missing_labels = 0
start_time = time.time()

for i, qid in enumerate(linked_qids):
    if (i + 1) % 5000 == 0:
        elapsed = time.time() - start_time
        rate = (i + 1) / elapsed if elapsed > 0 else 0
        remaining = (len(linked_qids) - (i + 1)) / rate if rate > 0 else 0
        print(f"{ts()} | [loading labels] {i + 1:,}/{len(linked_qids):,} @ {rate:.0f}/s | ~{format_time_remaining(remaining)} remaining")
    
    cache_file = os.path.join(CACHE_DIR, f"{qid}.json")
    if os.path.exists(cache_file):
        try:
            with open(cache_file, "r") as f:
                entity = json.load(f)
            
            labels = entity.get("labels", {})
            label = None
            if "en" in labels:
                label = labels["en"].get("value")
            elif labels:
                label = list(labels.values())[0].get("value")
            
            if label:
                labels_cache[qid] = label
            else:
                labels_cache[qid] = qid
                missing_labels += 1
        except:
            labels_cache[qid] = qid
            missing_labels += 1
    else:
        labels_cache[qid] = qid
        missing_labels += 1

elapsed_total = time.time() - start_time
print(f"{ts()} | [done] Loaded {len(labels_cache):,} labels ({missing_labels:,} missing, using Q-ID) in {format_time_remaining(elapsed_total)}")

# ---------------------------------------------------------------------------
# Helper functions
# ---------------------------------------------------------------------------

def get_label_fast(qid):
    """Get label from in-memory cache."""
    return labels_cache.get(qid, qid)

def normalize_label(label):
    """Normalize label to title case for consistency."""
    if not label:
        return label
    # Title case, but preserve all-caps acronyms
    return label[0].upper() + label[1:] if len(label) > 1 else label.upper()

def dedupe_labels_case_insensitive(labels_list):
    """
    Deduplicate labels that differ only by capitalization.
    Keeps the first occurrence's capitalization, normalized to title case.
    """
    seen = {}  # lowercase -> normalized label
    result = []
    
    for label in labels_list:
        if not label:
            continue
        lower = label.lower()
        if lower not in seen:
            # Normalize: capitalize first letter
            normalized = normalize_label(label)
            seen[lower] = normalized
            result.append(normalized)
    
    return result

def resolve_keyword(kw):
    """Resolve keyword like 'artist:Q5592' to 'artist:Rogier van der Weyden'."""
    if not kw or ":" not in kw:
        return None
    
    prefix, value = kw.split(":", 1)
    
    if not prefix or not value:
        return None
    
    if value.startswith("Q"):
        label = get_label_fast(value)
        if label:
            label = normalize_value(label)
            label = normalize_label(label)
            # Clean movement labels
            if prefix == "movement":
                label = clean_movement_label(label)
        if not label:
            label = value
        return f"{prefix}:{label}"
    
    return kw

# ---------------------------------------------------------------------------
# Check current state
# ---------------------------------------------------------------------------

print(f"\n{ts()} | Checking current state...")

painting_qids_set = set(painting_qids)
image_files = [f for f in os.listdir(IMAGES_DIR) if f.lower().endswith(".jpg")]
image_qids = set(os.path.splitext(f)[0] for f in image_files)

print(f"{ts()} | Paintings in query: {len(painting_qids_set):,}")
print(f"{ts()} | Images in folder: {len(image_qids):,}")
print(f"{ts()} | Images matching query: {len(image_qids & painting_qids_set):,}")
print(f"{ts()} | Images NOT in query: {len(image_qids - painting_qids_set):,}")

# ---------------------------------------------------------------------------
# Filter records that have image and year
# ---------------------------------------------------------------------------

print(f"\n{ts()} | Filtering records with image and year...")

eligible_records = []
no_record = 0
no_image = 0
no_year = 0

for qid in painting_qids:
    uri = f"https://www.wikidata.org/wiki/{qid}"
    
    if uri not in records:
        no_record += 1
        continue
    
    rec = records[uri]
    img_path = os.path.join(IMAGES_DIR, f"{qid}.jpg")
    year = rec.get("year")
    
    if not os.path.exists(img_path):
        no_image += 1
        continue
    
    if year is None or year == "":
        no_year += 1
        continue
    
    eligible_records.append(rec)

print(f"{ts()} | Found {len(eligible_records):,} eligible records")
print(f"{ts()} | Skipped: {no_record:,} no record, {no_image:,} no image, {no_year:,} no year")

# ---------------------------------------------------------------------------
# Resolve labels for keywords and claim fields
# ---------------------------------------------------------------------------

print(f"\n{ts()} | Resolving labels...")

total = len(eligible_records)
start_time = time.time()

for i, rec in enumerate(eligible_records):
    if (i + 1) % 2000 == 0 or i == 0:
        elapsed = time.time() - start_time
        rate = (i + 1) / elapsed if elapsed > 0 else 0
        remaining = (total - (i + 1)) / rate if rate > 0 else 0
        print(f"{ts()} | [resolving] {i + 1:,}/{total:,} @ {rate:.0f}/s | ~{format_time_remaining(remaining)} remaining")
    
    # Convert keywords set to sorted string with resolved labels
    resolved = []
    for kw in rec["keywords"]:
        r = resolve_keyword(kw)
        if r and not r.endswith(":"):
            resolved.append(r)
    
    # Deduplicate keywords by case-insensitive comparison
    seen_keywords = {}
    unique_keywords = []
    for kw in resolved:
        lower = kw.lower()
        if lower not in seen_keywords:
            seen_keywords[lower] = kw
            unique_keywords.append(kw)
    
    rec["keywords"] = ",".join(sorted(unique_keywords)) if unique_keywords else ""

    # Replace QIDs with labels in claim fields (with case-insensitive deduplication)
    for field in ["_artist", "_depicts", "_material", "_genre", "_movement", "_museum", "_country"]:
        qids_str = rec.get(field, "")
        if qids_str:
            labels = []
            for q in qids_str.split(","):
                q = q.strip()
                if q.startswith("Q"):
                    label = get_label_fast(q)
                    if label:
                        label = normalize_value(label)
                        # Clean movement labels
                        if field == "_movement":
                            label = clean_movement_label(label)
                    if not label:
                        label = q
                    labels.append(label)
                elif q:
                    labels.append(q)
            
            # Deduplicate labels that differ only by capitalization
            deduped = dedupe_labels_case_insensitive(labels)
            rec[field] = ", ".join(deduped)

elapsed_total = time.time() - start_time
print(f"{ts()} | [done] Resolved labels in {format_time_remaining(elapsed_total)}")

# ---------------------------------------------------------------------------
# Write CSV
# ---------------------------------------------------------------------------

print(f"\n{ts()} | Writing CSV...")

written = 0
written_qids = set()
skipped_no_keywords = 0

with open(csv_path, "w", newline="", encoding="utf-8") as fh:
    writer = csv.DictWriter(fh, fieldnames=FIELDNAMES, quoting=csv.QUOTE_MINIMAL)
    writer.writeheader()
    
    for rec in eligible_records:
        qid = rec["id"]
        keywords = rec.get("keywords", "")
        
        if not keywords:
            skipped_no_keywords += 1
            continue
        
        writer.writerow({fn: rec.get(fn, "") for fn in FIELDNAMES})
        written += 1
        written_qids.add(qid)

csv_size = os.path.getsize(csv_path)
print(f"{ts()} | CSV written: {written:,} artworks, {csv_size / 1024 / 1024:.1f} MB → {csv_path}")
print(f"{ts()} | Skipped {skipped_no_keywords:,} records with no keywords")

# ---------------------------------------------------------------------------
# Move images NOT in painting_qids OR not written to CSV
# ---------------------------------------------------------------------------

print(f"\n{ts()} | Cleaning up images...")

os.makedirs(SKIPPED_DIR, exist_ok=True)

moved = 0
kept = 0

for fname in image_files:
    qid = os.path.splitext(fname)[0]
    src = os.path.join(IMAGES_DIR, fname)
    
    if qid in painting_qids_set and qid in written_qids:
        kept += 1
        continue
    
    dst = os.path.join(SKIPPED_DIR, fname)
    
    if os.path.exists(dst):
        base, ext = os.path.splitext(fname)
        dst = os.path.join(SKIPPED_DIR, f"{base}__dup{ext}")
    
    shutil.move(src, dst)
    moved += 1

print(f"{ts()} | Kept {kept:,} images, moved {moved:,} to {SKIPPED_DIR}")

# ---------------------------------------------------------------------------
# Summary
# ---------------------------------------------------------------------------

print(f"\n{'='*60}")
print(f"SUMMARY")
print(f"{'='*60}")
print(f"Paintings in query:      {len(painting_qids):,}")
print(f"Records built:           {len(records):,}")
print(f"Eligible (image+year):   {len(eligible_records):,}")
print(f"Skipped (no keywords):   {skipped_no_keywords:,}")
print(f"Written to CSV:          {written:,}")
print(f"CSV size:                {csv_size / 1024 / 1024:.1f} MB")
print(f"Images kept:             {kept:,}")
print(f"Images moved to skipped: {moved:,}")
print(f"Output: {csv_path}")
print(f"{'='*60}")

In [ ]:
# 6. RUN vikus-viewer-script to create textures and spritesheet assets
# see: https://github.com/cpietsch/vikus-viewer-script

# !vikus-viewer-script "./data/images/*.jpg"

!cd vv; vikus-viewer-script "./data/images/*.jpg" --textureFormat avif --textureQuality 50 --spriteFormat avif --spriteQuality 50 --spriteSize 96 --mediumSize 1024 --largeSize 4096